In [35]:
# Запустить в начале работы
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from datetime import datetime

import pandas as pd
import numpy as np
# from gensim.models import KeyedVectors

import os

from clickhouse_driver import Client
cl_client = Client('localhost')

### Данные в базе ClickHouse

В базу ClickHouse данные предварительно заносятся следующие данные:

- model: категория (модель адео), которую купили
- pair: категория (модель адео), которую купил один и тот же клиент
- distance: сколько дней прошло между покупками

Пример:

Клиент совершил 3 покупки:

| Клиент ID | Дата       | Товары (категории) |   |   |
|-----------|------------|--------------------|---|---|
| 1         | 01.01.2020 | A,B                |   |   |
| 1         | 10.01.2020 | C,D                |   |   |
| 1         | 15.01.2020 | E,D                |   |   |


В ClickHouse данные будут внесены следующим образом.   
Считаем расстояние в днях между покупками.
Прошу обратить внимание, что учитывается ближайшая покупка.  
Например, товар D присутствует в 2х чеках. Расстояние считается для ближайшего чека (например, A-D - 10 дней)

| Запись | model | pair | distance |  |  |
|-|-|-|-|-|-|
| 1 | A | B | 0 |  |  |
| 2 | A | C | 10 |  |  |
| 3 | A | D | 10 |  |  |
| 4 | A | E | 15 |  |  |
| 5 | B | A | 0 |  |  |
| 6 | B | C | 10 |  |  |
| 7 | B | D | 10 |  |  |
| 8 | B | E | 15 |  |  |
| 9 | C | A | -10 |  |  |
| 10 | C | B | -10 |  |  |
| 11 | C | D | 0 |  |  |
| 12 | C | E | 5 |  |  |
| 13 | D | A | -10 |  |  |
| 14 | D | B | -10 |  |  |
| 15 | D | C | 0 |  |  |
| 16 | D | E | 0 |  |  |
| 17 | E | A | -15 |  |  |
| 18 | E | B | -15 |  |  |
| 19 | E | C | -5 |  |  |
| 20 | E | D | 0 |  |  |


### Извлечение данных для анализа

За год у меня получилось около 5млрд записей. Можно теперь их различным образом агрегировать.   
К сожалению, не визуализирована таблица с нужным коэффициентом, но в коде ниже показано, как он считается.  

В примере ниже ключевыми параметрами являются:
- count - число покупок товаров вместе
- quantile25 - нижний квартиль параметра distanse
- quantile75 - верхний квартиль параметра distanse

Квантили считаются в целях исключения выбросов.  
*Прошу учесть, что можно выбрать другие персентили, например 10 и 90, или 20 и 80 - это подбирается уже опытным путем, 25 и 75 показали неплохой результат.  

Далее комментарии в самом коде. 


In [1]:
def load_model(model):    
    resp = cl_client.execute(
       "SELECT model, pair, count(distance), avg(distance), median(distance), stddevPop(distance), quantiles(0.25, 0.75)(distance) " \
       "FROM models " \
       "WHERE distance < 45 and distance > -45 and model=='{}' " \
       "GROUP BY model, pair " \
       "ORDER BY model, count(distance) DESC".format(model))
    df = pd.DataFrame(list(resp), columns=['model', 'pair', 'count','avg','median', 'stdev', 'quantiles'])
    
    df['quantile25'] = df['quantiles'].apply(lambda x: x[0])
    df['quantile75'] = df['quantiles'].apply(lambda x: x[1])
    
    # Считаем параметр scatter (разброс). Он говорит, какой разброс в днях между покупками товаров.  
    # Чем ниже этот параметр, тем больше тяготеет покупка одного товара к другому.  
    df['scatter'] = df['quantile75'] - df['quantile25']
    df = df.drop(columns=['quantiles'])
    df = df.merge(models, left_on='model', right_on='ModelAdeo').merge(models, left_on='pair', right_on='ModelAdeo').drop(columns=['ModelAdeo_x', 'ModelAdeo_y'])
    
    # Делим число покупок на показатель разброса. 
    # Получаем соответстующий коэффициент, который используется в дальнейшем анализе.
    df['koef'] = df['count'] / (df['scatter'] + 1)
    return df

In [44]:

#  Эту таблицу можно не смотреть, в ней нет нужного коэффициента.
load_model('MOD_201189')[['model_name_x', 'model_name_y', 'count', 'avg', 'median', 'stdev']].sort_values(by='count', ascending=False).head(60)


,model_name_x,model_name_y,count,avg,median,stdev
0,Водяной радиатор,Полипропиленовые фитинги,59390,2.243980,0.0,15.666916
1,Водяной радиатор,Муфта переходная,44678,0.495053,0.0,5.410774
2,Водяной радиатор,Монтажные и строительные клеи,43238,2.165618,0.0,20.755092
3,Водяной радиатор,Крепление радиатора,37541,0.964226,0.0,7.214107
4,Водяной радиатор,ППР труба,33169,1.310923,0.0,14.610811
5,Водяной радиатор,Лампочка,32173,3.483915,0.0,21.779439
6,Водяной радиатор,запорная арматура,31849,1.199786,0.0,13.435963
7,Водяной радиатор,Шуруп для гипсокартона,30752,1.167794,0.0,21.399368
8,Водяной радиатор,Монтажная пена,28872,2.173975,0.0,20.953323
9,Водяной радиатор,Перчатки,27439,0.748861,0.0,21.377822


In [6]:
tmp = []

for model in list(models['ModelAdeo'].unique()):
    df = load_model(model)
    tmp.append(df)
    print(model)
    

MOD_200001
MOD_200002
MOD_200003
MOD_200004
MOD_200005
MOD_200006
MOD_200007
MOD_200008
MOD_200009
MOD_200010
MOD_200011
MOD_200012
MOD_200013
MOD_200014
MOD_200015
MOD_200016
MOD_200017
MOD_200018
MOD_200019
MOD_200020
MOD_200021
MOD_200022
MOD_200023
MOD_200024
MOD_200025
MOD_200026
MOD_200027
MOD_200028
MOD_200029
MOD_200030
MOD_200031
MOD_200032
MOD_200033
MOD_200034
MOD_200035
MOD_200036
MOD_200037
MOD_200038
MOD_200039
MOD_200040
MOD_200041
MOD_200042
MOD_200043
MOD_200045
MOD_200046
MOD_200047
MOD_200048
MOD_200049
MOD_200050
MOD_200051
MOD_200052
MOD_200053
MOD_200054
MOD_200055
MOD_200056
MOD_200057
MOD_200058
MOD_200059
MOD_200060
MOD_200061
MOD_200062
MOD_200063
MOD_200064
MOD_200065
MOD_200066
MOD_200067
MOD_200068
MOD_200069
MOD_200070
MOD_200071
MOD_200072
MOD_200073
MOD_200074
MOD_200075
MOD_200076
MOD_200077
MOD_200078
MOD_200079
MOD_200080
MOD_200081
MOD_200082
MOD_200083
MOD_200084
MOD_200085
MOD_200086
MOD_200087
MOD_200088
MOD_200089
MOD_200090
MOD_200091
MOD_200092

KeyboardInterrupt: 

In [79]:
df = pd.concat(tmp)
df = df.merge(models, left_on='model', right_on='ModelAdeo').merge(models, left_on='pair', right_on='ModelAdeo').drop(columns=['ModelAdeo_x', 'ModelAdeo_y'])
print(df.shape)
df.head()


(0, 12)


,model,pair,count,avg,median,std,quantile25,quantile75,scatter,art_median,Name_x,Name_y


In [83]:
"SELECT model, pair, count(distance), avg(distance), median(distance), stddevPop(distance), quantiles(0.25, 0.75)(distance) " \
       "FROM models " \
       "WHERE distance < 45 and distance > -45 and model=='{}' " \
       "GROUP BY model, pair " \
       "ORDER BY model, count(distance) DESC".format(123)

"SELECT model, pair, count(distance), avg(distance), median(distance), stddevPop(distance), quantiles(0.25, 0.75)(distance) FROM models WHERE distance < 45 and distance > -45 and model=='123' GROUP BY model, pair ORDER BY model, count(distance) DESC"